## Code from Here!!

In [2]:
import pandas as pd
import os

# loop all files in the current folder
movies = [mo for mo in os.listdir(os.getcwd()) if 'csv' in mo and 'movie list' not in mo]
len(movies)

1916

In [1]:
# clean the comment and generate a new dataframe
import re

def comment_clean(cmt, movie_name):
    cmt_list = re.split(r"\", u'|\', u'|\.000Z", cmt.replace(' [],', '').replace('[', '').replace(']', ''))
    cmt_list_strip = [(comment.lstrip("\"\'").lstrip("\" u\'").lstrip("\' \"").lstrip("\"', '").rstrip("\'\"")
                      .rstrip("\"\'").replace("\\n","")).replace("\'\\\', \"", '')
                      for comment in cmt_list]
    cmt_list_clean = [item for item in cmt_list_strip][:-1]
    cmt_content = []
    cmt_time = []
    if len(cmt_list_clean)%2 != 0:
        raise Exception('incorrect length')
    for ind in range(0,len(cmt_list_clean),2):
        cmt_content.append(cmt_list_clean[ind])
        cmt_time.append(cmt_list_clean[ind+1])
    cmt_dict = dict(movie = movie_name, comment = cmt_content, time = cmt_time)
    cmt_df = pd.DataFrame(cmt_dict)
    cmt_df['time'] = pd.to_datetime(cmt_df['time'].apply(lambda x: x.replace('.000Z','')), format = '%Y-%m-%dT%H:%M:%S')
    return cmt_df


def movie_process(movie_name, selected_cols = ['countComments', 'countViews', 'countLikes', 'countDislikes']):
#     get clean movie name and read in table
    movie_name_clean = movie_name.split(' 20')[0]
    cur_movie = pd.read_csv(movie_name)
    if cur_movie.shape[0] == 0:
        return None, None
#     extract comment data
    cmt_df = None
    for cmt in cur_movie['Comments']:
        try:
            cmt_df_tmp = comment_clean(cmt, movie_name_clean)
            if cmt_df is None:
                cmt_df = cmt_df_tmp
            else:
                cmt_df = cmt_df.append(cmt_df_tmp)
        except:
            print movie_name, cmt[0:10]
            raise Exception('Unexpected comment format, check the comment content')
    
#     sum movie data
    cur_movie_select = cur_movie[selected_cols]
    cur_movie_select = cur_movie_select.assign(movie_title = movie_name_clean)
    cur_movie_sum = cur_movie_select.groupby('movie_title', as_index = False).sum()
#     cur_movie_sum = cur_movie_sum.assign(Dimension = cur_movie['Dimension'].unique())
    
    return cur_movie_sum, cmt_df

In [29]:
movie_data = None
cmt_data = None
for movie_name in movies:
    mov_tmp, cmt_tmp = movie_process(movie_name)
    if movie_data is None:
        movie_data = mov_tmp
    else:
        movie_data = movie_data.append(mov_tmp)
    if cmt_data is None:
        cmt_data = cmt_tmp
    else:
        cmt_data = cmt_data.append(cmt_tmp)

In [280]:
# write to output folder
if not os.path.exists('output'):
    os.mkdir('output')
cmt_data.to_csv('output/comments.csv', index=None)


In [30]:
movie_data.to_csv('output/movies.csv', index=None)

## processing

In [42]:
trailer_data = pd.read_csv("C:/Users/oweni/Desktop/big data/youtube/output/movies.csv")
performance_data = pd.read_csv("C:/Users/oweni/Desktop/big data/youtube/output/boxofficefinal.csv",index_col = "Movie")

In [36]:
performance_data.head()

,ID,Release Date,Movie,Production Budget,Domestic Gross,International Gross,WorldWide Gross,Domestic Blu-Ray Sales,Domestic DVD Sales,Total Domestic Video Sales,Genre,MPAA Rating,Movie Studio (Distributor)
0,1,12/18/2009,Avatar,"$425,000,000","$760,507,625","$2,023,411,357","$2,783,918,982","$162,034,656","$232,820,270","$394,854,926",Action,PG-13,20th Century Fox
1,2,12/18/2015,Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$1,122,000,000","$2,058,662,225","$139,743,772","$32,462,937","$172,206,709",Adventure,PG-13,Walt Disney
2,3,5/24/2007,Pirates of the Caribbean: At World�s End,"$300,000,000","$309,420,425","$654,000,000","$963,420,425","$6,123,101","$312,315,495","$318,438,596",Adventure,PG-13,Walt Disney
3,4,11/6/2015,Spectre,"$300,000,000","$200,074,175","$679,546,748","$879,620,923","$23,637,497","$12,199,184","$35,836,681",Action,PG-13,Sony Pictures
4,5,7/20/2012,The Dark Knight Rises,"$275,000,000","$448,139,099","$636,300,000","$1,084,439,099","$80,198,499","$71,782,028","$151,980,527",Action,PG-13,Warner Bros


In [37]:
trailer_data.head()

,movie_title,countComments,countViews,countLikes,countDislikes
0,#Horror,8629,23452637,86772,4387
1,10 Cloverfield Lane,8148,9923330,32396,1211
2,10 Days in a Madhouse,41,56554,214,24
3,10th & Wolf,30,97035,140,5
4,12 Rounds,14,69078,79,11


In [46]:
full_data = trailer_data.join(performance_data, on=['movie_title'],how='inner')

In [47]:
full_data

,movie_title,countComments,countViews,countLikes,countDislikes,ID,Release Date,Production Budget,Domestic Gross,International Gross,WorldWide Gross,Domestic Blu-Ray Sales,Domestic DVD Sales,Total Domestic Video Sales,Genre,MPAA Rating,Movie Studio (Distributor)
0,#Horror,8629,23452637,86772,4387,4697,11/20/2015,"$1,500,000",$0,$0,$0,NaN,NaN,NaN,Horror,Not,NaN
1,10 Cloverfield Lane,8148,9923330,32396,1211,3909,3/11/2016,"$5,000,000","$72,082,999","$36,203,423","$108,286,422","$4,064,827","$3,215,156","$7,279,983",Thriller/Suspense,PG-13,Paramount Pictures
2,10 Days in a Madhouse,41,56554,214,24,3177,11/11/2015,"$12,000,000","$14,616",$0,"$14,616",NaN,NaN,NaN,Drama,R,Cafe Pictures
3,10th & Wolf,30,97035,140,5,3611,8/18/2006,"$8,000,000","$54,702","$89,080","$143,782",NaN,NaN,NaN,Drama,Not,inkFilm
4,12 Rounds,14,69078,79,11,2410,3/27/2009,"$20,000,000","$12,234,694","$5,071,954","$17,306,648",NaN,"$9,077,332","$9,077,332",Action,PG-13,20th Century Fox
5,12 Years a Slave,8892,6673173,15263,900,2300,10/18/2013,"$20,000,000","$56,671,993","$124,353,350","$181,025,343","$7,716,085","$16,848,324","$24,564,409",Drama,R,Fox Searchlight
6,127 Hours,9098,12634126,28113,669,2568,11/5/2010,"$18,000,000","$18,335,230","$41,881,941","$60,217,171","$2,781,183","$10,680,659","$13,461,842",Drama,R,Fox Searchlight
8,1408,601,4030368,6086,291,2195,6/22/2007,"$22,500,000","$71,985,628","$59,277,742","$131,263,370",NaN,"$49,766,172","$49,766,172",Horror,R,MGM
9,16 Blocks,52,366365,438,45,1180,3/3/2006,"$45,000,000","$36,895,141","$28,700,000","$65,595,141",NaN,"$20,191,475","$20,191,475",Action,PG-13,Warner Bros
10,16 to Life,102,142157,411,29,4856,9/3/2010,"$1,000,000","$11,574",$0,"$11,574",NaN,NaN,NaN,Romantic Comedy,Not,NaN


In [48]:
full_data.to_csv('output/full.csv', index=None)